In [1]:
import pandas as pd
from scipy.optimize import minimize_scalar

import json
import requests
import ast
from tqdm import tqdm
import io
from io import StringIO

import oci
import ads

In [2]:
ads.set_auth("resource_principal") # a signer for all ads operations, managed automatically
auth = oci.auth.signers.get_resource_principals_signer()
object_storage = oci.object_storage.ObjectStorageClient({}, signer=auth)

# Import Data and Model

### Import new cases csv

In [3]:
namespace = object_storage.get_namespace().data
bucket_name='filesdemo'

In [ ]:
new_cases_file= 'operational_research/new_cases.csv'

obj = object_storage.get_object(namespace, bucket_name, new_cases_file)
df = pd.read_csv(io.BytesIO(obj.data.content))
df.columns=['id','age','risk']

### Import parameters

In [4]:
parameters_file= 'operational_research/optimization_parameters.txt'

parameters = object_storage.get_object(
    namespace_name=namespace,
    bucket_name=bucket_name,
    object_name=parameters_file
)

params_txt = parameters.data.content.decode("utf-8")
params = ast.literal_eval(params_txt)

lambda_star = params["lambda_star"]
cost = params["cost"]
p_min = params["p_min"]
p_max = params["p_max"]

### Demand model endpoint

In [5]:
endpoint = '<your endpoint>'

# Individual Optimization of New Cases

In [7]:
def predict_demand_batch(df_row):
    body = {
        "data_type": "pandas.core.frame.DataFrame",
        "data": df_row.to_json(orient='records')
    }
    response = requests.post(endpoint, json=body, auth=auth)
    predictions = response.json()['prediction']
    return predictions

In [8]:
def inner_objective(price, age, risk):
    X_tmp = pd.DataFrame([[price, age, risk]], columns=['price', 'age', 'risk'])
    
    d = predict_demand_batch(X_tmp)[0]
    return -(d * (price - cost) + lambda_star * d)

In [9]:
def optimal_price(age, risk):
    res = minimize_scalar(
        inner_objective,
        bounds=(p_min, p_max),
        args=(age, risk),
        method="bounded",
        options={"xatol": 1e-4}
    )
    return res.x

In [10]:
def compute_optimal_prices(df_new):
    results = []
    
    for idx, row in tqdm(df_new.iterrows(), total=len(df_new), desc="ind optimzation"):
        customer_id = row['id']
        age = row['age']
        risk = row['risk']
        
        opt_price = optimal_price(
            age=age,
            risk=risk,
        )
        
        results.append({
            'id': customer_id,
            'optimal_price': opt_price
        })
    
    return pd.DataFrame(results)

In [11]:
optimal_prices=compute_optimal_prices(df)

ind optimzation: 100%|██████████| 50/50 [00:39<00:00,  1.28it/s]

      id  optimal_price
0    0.0       3.898726
1    1.0       5.999938
2    2.0       3.617225
3    3.0       5.999938
4    4.0       3.942123
5    5.0       5.999938
6    6.0       5.999938
7    7.0       3.664838
8    8.0       5.999938
9    9.0       5.999938
10  10.0       5.999938
11  11.0       5.999938
12  12.0       5.999938
13  13.0       5.999938
14  14.0       3.889174
15  15.0       5.999938
16  16.0       3.948781
17  17.0       5.999938
18  18.0       5.999938
19  19.0       5.999938
20  20.0       5.999938
21  21.0       3.982222
22  22.0       5.999938
23  23.0       5.999938
24  24.0       5.999938
25  25.0       5.999938
26  26.0       5.999938
27  27.0       5.999938
28  28.0       3.740368
29  29.0       5.999938
30  30.0       5.999938
31  31.0       5.999938
32  32.0       5.999938
33  33.0       3.953262
34  34.0       5.999938
35  35.0       5.999938
36  36.0       3.573766
37  37.0       3.818489
38  38.0       5.999938
39  39.0       5.999938
40  40.0       3

# Export prices files to Object Storage

In [13]:
object_name = "operational_research/optimal_prices.csv"

csv_buffer = StringIO()
optimal_prices.to_csv(csv_buffer, index=False)

object_storage.put_object(
    namespace_name=namespace,
    bucket_name=bucket_name,
    object_name=object_name,
    put_object_body=csv_buffer.getvalue()
)